In [6]:
import sys
import os
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)
from hpelm import ELM
import utils.NN_building as NN_building
import pdb
import tqdm
import numpy as np
from sklearn.metrics import accuracy_score  
from matplotlib import pyplot as plt    
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 

In [2]:
parameters = {
    'model_type': 'Naive',
    'use_trap_info': True,
    'ntraps': 11,
    'lags': 5,
    'random_split': False,
    'test_size': 0.2,
    'scale': False,
    'input_3d': False,
    'bool_input': False,
    'truncate_100': True,
    'cylindrical_input': True,
    'add_constant': True
    }

data_path = f"../results/final_dfs/final_df_lag{parameters['lags']}_ntraps{parameters['ntraps']}.parquet"

# create dataset
x_train, x_test, y_train, y_test, nplaca_index = NN_building.create_dataset(parameters,data_path)

# Network structure
model_input, model_output = NN_building.input_output_sizes(x_train, parameters['model_type'])


In [3]:
scaler = MinMaxScaler(feature_range=(-1, 1))
x_train_norm = scaler.fit_transform(x_train)
x_test_norm = scaler.transform(x_test)

In [9]:
2**20

1048576

In [ ]:
y_train_onehot = np.zeros((y_train.size, y_train.max() + 1))
y_train_onehot[np.arange(y_train.size), y_train] = 1

acc_train = []
acc_test = []

index = []


for j in tqdm.tqdm([20]):
    for k in [1.2]:#[0.8,0.9,1,1.1,1.2]:    
        p = int(np.ceil((2**j)*k))
        if p in index:
            continue
        row_train = []
        row_test = []
        for i in range(1):
            elm = ELM(x_train_norm.shape[1], y_train_onehot.shape[1], classification="c")
            elm.add_neurons(p, "sigm")
            elm.train(x_train_norm, y_train_onehot, "c")
            y_hat = elm.predict(x_test_norm).argmax(axis=1)
            y_hat_train = elm.predict(x_train_norm).argmax(axis=1)
            row_train.append(accuracy_score(y_train, y_hat_train))
            row_test.append(accuracy_score(y_test, y_hat))
        index.append(p)
        acc_train.append(row_train)
        acc_test.append(row_test)
        pd.DataFrame(index).to_csv('index.csv')
        pd.DataFrame(acc_train).to_csv('acc_train.csv')
        pd.DataFrame(acc_test).to_csv('acc_test.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
acc_train = np.array(acc_train) 
acc_test = np.array(acc_test)
acc_train_mean = acc_train.mean(axis=1)
acc_test_mean = acc_test.mean(axis=1)
plt.plot(acc_train_mean)
plt.plot(acc_test_mean)

NameError: name 'np' is not defined